# Synchronous Interlock CCW/Rotator Integration Test

This notebook performs a synchronous motion interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

Warning! The current MTRotator (Jan 2022) release does not support the clearError command anymore. This happens now automatically as part of enabling the CSC.

Warning! Revise the way how velocities are set. This is also under development.

In [ ]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [ ]:
test_message = "Interlock Rotator_CCW Integration Test - LVV-T1569"

In [ ]:
test_message = "Interlock Rotator_CCW Integration Test - limit switch characterisation"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "MTRotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

Enable the Pointing component:

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

Rotator-Mount start-up sequence:

In [ ]:
await rot.cmd_start.set_start(timeout=10)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
Rotator state machine commands: (only use when needed)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.DISABLED)                             

In [ ]:
await rot.cmd_start.set_start(timeout=10)

In [ ]:
await rot.cmd_enable.set_start(timeout=10)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.STANDBY)        

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.OFFLINE)

Mount state machine commands:

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

In [ ]:
#await salobj.set_summary_state(mtm, salobj.State.OFFLINE)

# CCW check:
The loop is automatically started when the CSC is enabled, but it will fail if it can’t get data from the rotator. The recommended way to bring up MTRotator and MTMount is to first put both in DISABLED state (in any order), then enable MTMount (so the CCW centers itself on the rotator) then enable MTRotator. (Thanks Russell for the explanation)

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)

In [ ]:
await mtm.cmd_enableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

In [ ]:
await mtm.cmd_disableCameraCableWrapFollowing.set_start(timeout=10.)

# Move to 0 deg Starting Position

In [ ]:
print("Test Start")

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to 0 deg Starting Position - {now} UTC")

print("Move to 0.0 deg starting position")

#await rot.cmd_configureVelocity.set_start(vlimit=1.0)
await rot.cmd_move.set_start(position=0.0,timeout=30.)

In [ ]:
alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_stop.start(timeout=10.)
await asyncio.sleep(5.)

In [ ]:
print("start tracking")
await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)

# Start Test

In [ ]:
print("Test Start")

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Positive Interlock Test - {now} UTC")


# Test the followErrorThreshold

Here the rotator is brought on purpuse over the follwoErrorThreshold and expected to fault

In [ ]:

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to 0 deg Starting Position - {now} UTC")


In [ ]:
await rot.cmd_configure.set_start(followingErrorThreshold=0.5)

In [ ]:
#await rot.cmd_configureVelocity.set_start(vlimit=3.5)

In [ ]:
await asyncio.sleep(1.)

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to X deg - {now} UTC")

print("Move to X deg position")
await rot.cmd_move.set_start(position=0.5,timeout=60.)

#print("Reset the Rotator and CCW")

Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearError.start(timeout=30)

Wait for Safety Reset

In [ ]:
await rot.cmd_clearError.start(timeout=30)

Re-enable the CCW

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_enableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

# Move to Zero

In [ ]:
print("Move to 0.0 deg starting position")

await rot.cmd_move.set_start(position=0.0,timeout=30.)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.STANDBY)

Wait for override off

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

# Set Low Velocity and Move Through Positive Interlock
This will move the rotator indepently from the CCW and will therefore activate the positive limit switch located at the bulkhead plate between the Rotator and the CCW

In [ ]:
#await rot.cmd_configureVelocity.set_start(vlimit=1.5)

Disable the CCW

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_disableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

This will make the rotator fault

In [ ]:
await asyncio.sleep(1.)

print("Move to +10 deg position")

await rot.cmd_move.set_start(position=+10,timeout=30.)

print("Reset the Rotator and CCW")

Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for CCW override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

Reactivate the CCW

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_enableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = await mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Set Low Velocity and Move Through Negative Interlock
This will move the rotator indepently from the CCW and will therefore activate the negative limit switch located at the bulkhead plate between the Rotator and the CCW

In [ ]:
#await rot.cmd_configureVelocity.set_start(vlimit=1.5)

Disable the CCW

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_disableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

This will make the rotator fault

In [ ]:
await asyncio.sleep(1.)

print("Move to +10 deg position")

await rot.cmd_move.set_start(position=+10,timeout=30.)

print("Reset the Rotator and CCW")

Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for CCW override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

Reactivate the CCW

In [ ]:
CCWstate = mtm.evt_cameraCableWrapFollowing.get()
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_enableCameraCableWrapFollowing.set_start(timeout=10.)
await CCWstate = await mtm.evt_cameraCableWrapFollowing.next(timeout=10.)
print("CCWstate:",CCWstate.enabled)

In [17]:
#Set speed back to original value
#await rot.cmd_configureVelocity.set_start(vlimit=3.5)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to Zero

In [ ]:
print("Move to 0.0 deg starting position")

await rot.cmd_move.set_start(position=0.0,timeout=30.)

# Test Complete

In [ ]:
print("Test complete")
now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message} - Positive Interlock Test - {now} UTC")

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for CCW override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_move.set_start(position=19.04,timeout=30.)

# Additional Error Handling Commands

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearError.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)